In [1]:
!pip install fastapi uvicorn wandb pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00


In [2]:
!ngrok authtoken 2oy6ThE6Tf1Sp0Mw5cqPqLKri4U_5yMGJddEa9wQrQs1ieQeC

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
%%writefile app.py

import os
import wandb
import sklearn
from joblib import load
from fastapi import FastAPI, HTTPException
from contextlib import asynccontextmanager
from pydantic import BaseModel
import wandb
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import joblib
from pyngrok import ngrok

app = FastAPI()

os.environ["WANDB_API_KEY"] = "6697a9d4d424fdce3f97021af11508c1941caf7e"

# Define input data schema


class PredictionRequest(BaseModel):
    song_name: str
    song_duration_ms: int
    acousticness: float
    danceability: float
    energy: float
    instrumentalness: float
    key: int
    liveness: float
    loudness: float
    audio_mode: int
    speechiness: float
    tempo: float
    time_signature: int
    audio_valence: float


# Load the model from WandB


def load_model_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB

        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        print(artifact)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        print(model_path)
        model = joblib.load(model_path)
        print(model)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")


def load_transformer_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB

        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        print(artifact)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        print(model_path)
        model = joblib.load(model_path)
        print(model)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")


project_name = "MLOPS_SONG_POPULARITY_PREDICTION"  # Replace with your project name
artifact_model_name = "Model_Song_Popularity:latest"  # Replace with your artifact name
ml_model = load_model_from_wandb(project_name, artifact_model_name)

artifact_transformer_name = (
    "Tranformer_Song_Popularity:latest"  # Replace with your artifact name
)
poly = load_transformer_from_wandb(project_name, artifact_transformer_name)

# Prediction endpoint


@app.post("/predict")
def predict(input_data: PredictionRequest):
    try:
        # Convert input data to a dictionary for prediction

        input_dict = input_data.dict()

        # Specify the columns to be used for prediction

        selected_features = [
            "acousticness",
            "danceability",
            "energy",
            "instrumentalness",
            "liveness",
            "loudness",
            "tempo",
            "audio_valence",
        ]

        # Filter the input dictionary to include only selected features

        filtered_data = {k: input_dict[k] for k in selected_features}

        df = pd.DataFrame(filtered_data, index=[0])

        # Apply the same polynomial transformation

        transformed_features = poly.transform(df)  # Transform the input data the same

        # Call the model's prediction method

        prediction = ml_model.predict(transformed_features)

        # Return the prediction result

        return {"Estimated Song Popularity": prediction[0]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")


Writing app.py


In [5]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 &

nohup: appending output to 'nohup.out'


In [6]:
!ps -ax | grep uvicorn

   1332 ?        Dl     0:04 /usr/bin/python3 /usr/local/bin/uvicorn app:app --host 0.0.0.0 --port 8
   1361 ?        S      0:00 /bin/bash -c ps -ax | grep uvicorn
   1363 ?        S      0:00 grep uvicorn


In [7]:
!tail nohup.out

wandb: 🚀 View run prime-gorge-215 at: https://wandb.ai/anmol_chhabra_ampba2025s-indian-school-of-business/MLOPS_SONG_POPULARITY_PREDICTION/runs/0b4fxut9
wandb: ⭐️ View project at: https://wandb.ai/anmol_chhabra_ampba2025s-indian-school-of-business/MLOPS_SONG_POPULARITY_PREDICTION
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20241215_142043-0b4fxut9/logs
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /content/wandb/run-20241215_142047-9popv6an
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run unique-sunset-216
wandb: ⭐️ View project at https://wandb.ai/anmol_chhabra_ampba2025s-indian-school-of-business/MLOPS_SONG_POPULARITY_PREDICTION
wandb: 🚀 View run at https://wandb.ai/anmol_chhabra_ampba2025s-indian-school-of-business/MLOPS_SONG_POPULARITY_PREDICTION/runs/9popv6an


In [8]:
# Expose the FastAPI app

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://10d0-35-245-92-199.ngrok-free.app" -> "http://localhost:8000"


Invoking the Prediction Service

In [9]:
import requests, json

In [10]:
data = """{ "song_name":"Boulevard of Broken Dreams",
            "song_duration_ms":262333,
            "acousticness":0.00552,
            "danceability":0.496,
            "energy":0.682,
            "instrumentalness":0.0000294,
            "key":8,
            "liveness":0.0589,
            "loudness":-4.095,
            "audio_mode":1,
            "speechiness":0.0294,
            "tempo":167.06,
            "time_signature":4,
            "audio_valence":0.474
          }
         """

In [11]:
json_data = data.replace('\n', '').replace(' ','')

In [12]:
json_data

'{"song_name":"BoulevardofBrokenDreams","song_duration_ms":262333,"acousticness":0.00552,"danceability":0.496,"energy":0.682,"instrumentalness":0.0000294,"key":8,"liveness":0.0589,"loudness":-4.095,"audio_mode":1,"speechiness":0.0294,"tempo":167.06,"time_signature":4,"audio_valence":0.474}'

In [14]:
response = requests.post('https://10d0-35-245-92-199.ngrok-free.app/predict', data=json_data)

In [15]:
print(response.json())

{'Estimated Song Popularity': 50.90268283212074}
